In [252]:
import geopandas as gpd
import pandas as pd
from eurocalliopelib.utils import eu_country_code_to_iso3

idx = pd.IndexSlice

FREIGHT_SECTORS = {
    'GT03': 'Mining and quarrying',
    'GT04': 'Food, beverages and tobacco',
    'GT05': 'Textiles and leather',
    'GT06': 'Wood and wood products',
    'GT11': 'Machinery Equipment',
    'GT12': 'Transport Equipment',
    'GT13': 'Other Industrial Sectors'
}

In [253]:
nuts_2006 = gpd.read_file("modules/industry/resources/eurostat/nuts_2006.geojson")
nuts_2006["nuts_id_eu"] = nuts_2006.NUTS_ID.str.replace("GR", "EL")
nuts_2006.head()

,NUTS_ID,LEVL_CODE,CNTR_CODE,NUTS_NAME,NAME_LATN,geometry,nuts_id_eu
0,DE80I,3,DE,Uecker-Randow,Uecker-Randow,"MULTIPOLYGON (((13.76656 53.68498, 13.76454 53...",DE80I
1,DE80C,3,DE,Müritz,Müritz,"POLYGON ((12.33095 53.31823, 12.34201 53.33172...",DE80C
2,UKD53,3,UK,Sefton,Sefton,"POLYGON ((-3.00874 53.43841, -3.00874 53.43921...",UKD53
3,DE804,3,DE,"Schwerin, Kreisfreie Stadt","Schwerin, Kreisfreie Stadt","POLYGON ((11.30194 53.65295, 11.30356 53.65872...",DE804
4,UKD32,3,UK,Greater Manchester North,Greater Manchester North,"POLYGON ((-2.48971 53.46017, -2.48984 53.46225...",UKD32


In [254]:

def read_eurostat_tsv(path_to_tsv, initial_columns):

    df = pd.read_csv(path_to_tsv, delimiter="\t")
    
    separate_cols = df.pop(df.columns[0]).str.split(",", expand=True)
    separate_cols.columns = initial_columns

    df.columns = df.columns.astype(int)

    def _ensure_numeric_values(series):
        series = series.astype(str).str.extract(r"(\-*\d+\.*\d*)")[0]
        return pd.to_numeric(series, errors='coerce')
    df = df.apply(_ensure_numeric_values)
    
    return pd.concat([separate_cols, df], axis="columns")


In [255]:
attr_columns = ["NST_2007_code", "units", "nuts3"]
freight = read_eurostat_tsv("modules/industry/resources/eurostat/freight.tsv.gz", attr_columns)
years = sorted([col for col in freight.columns if col not in attr_columns])
freight = freight[attr_columns + years]

# Imputation of missing values
freight[years] = freight[years].ffill(axis="columns").bfill(axis="columns")

# Add extra columns
freight["nuts2"] = freight["nuts3"].str[:-1]
freight["nuts1"] = freight["nuts2"].str[:-1]

In [256]:
# Get compatible shapes and desired industry categories
subregions = set(freight["nuts3"].unique()) & set(nuts_2006.NUTS_ID.unique())
freight_match = freight[freight['nuts3'].isin(list(subregions))]
freight_idees = freight_match[freight_match["NST_2007_code"].isin(FREIGHT_SECTORS)].copy()
freight_idees["IDEES_category"] = freight_idees["NST_2007_code"].replace(FREIGHT_SECTORS)

# Operation-ready dataframe
# freight_idees = freight_idees.drop(["NST_2007_code", "units"], axis='columns').set_index(["nuts1", "nuts2", "nuts3", "IDEES_category"])


In [257]:
freight_idees

,NST_2007_code,units,nuts3,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,nuts2,nuts1,IDEES_category
3383,GT03,THS_T,AT111,456.0,572.0,245.0,604.0,639.0,448.0,334.0,516.0,622.0,540.0,658.0,493.0,AT11,AT1,Mining and quarrying
3384,GT03,THS_T,AT112,2687.0,2804.0,2365.0,2836.0,3697.0,2701.0,4154.0,2583.0,3589.0,2737.0,4043.0,3400.0,AT11,AT1,Mining and quarrying
3385,GT03,THS_T,AT113,1805.0,1281.0,2227.0,1760.0,1526.0,1869.0,1467.0,1361.0,1332.0,1539.0,1553.0,1255.0,AT11,AT1,Mining and quarrying
3386,GT03,THS_T,AT121,6376.0,6340.0,5505.0,5530.0,5152.0,5167.0,6313.0,7372.0,6544.0,7889.0,9375.0,7290.0,AT12,AT1,Mining and quarrying
3387,GT03,THS_T,AT122,5696.0,4507.0,5764.0,6553.0,3908.0,5039.0,5960.0,6026.0,7613.0,7057.0,8222.0,9023.0,AT12,AT1,Mining and quarrying
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23090,GT13,THS_T,UKN01,215.0,53.0,62.0,198.0,216.0,75.0,192.0,66.0,43.0,115.0,115.0,115.0,UKN0,UKN,Other Industrial Sectors
23091,GT13,THS_T,UKN02,65.0,84.0,124.0,9.0,52.0,34.0,13.0,5.0,45.0,32.0,32.0,32.0,UKN0,UKN,Other Industrial Sectors
23092,GT13,THS_T,UKN03,140.0,98.0,50.0,64.0,24.0,35.0,31.0,44.0,37.0,100.0,100.0,100.0,UKN0,UKN,Other Industrial Sectors
23093,GT13,THS_T,UKN04,26.0,45.0,45.0,53.0,10.0,10.0,2.0,1.0,25.0,14.0,14.0,14.0,UKN0,UKN,Other Industrial Sectors


In [258]:
attr_columns = ['NACE_code', 'indicator', 'nuts']
activity_codes = pd.read_csv(
        "modules/industry/workflow/internal/industry_activity_codes.csv", skipfooter=7, index_col=0, header=0, engine='python'
    ).dropna(subset=['Eurostat sector'])

employees = read_eurostat_tsv(
    "modules/industry/resources/eurostat/employees.tsv.gz", attr_columns
)
employees = employees[employees["indicator"] == "V16110"]
years = sorted([col for col in employees.columns if col not in attr_columns])
employees = employees[attr_columns + years]

# Imputation of missing values
employees[years] = employees[years].ffill(axis="columns").bfill(axis="columns")

# Get desired industry categories
employees = employees[employees["NACE_code"].isin(activity_codes.index)]
employees["IDEES_category"] = employees['NACE_code'].replace(activity_codes["Eurostat sector"].to_dict())

# Get match with freight shapes
employees = employees[(employees["nuts"].isin(freight.nuts2))]
employees = employees.rename(columns={"nuts": "nuts2"})
employees["nuts1"] = employees["nuts2"].str[:-1]


In [274]:
employees_idx = employees.groupby(["nuts1", "nuts2", "IDEES_category"]).sum(numeric_only=True)
employees_idx.columns.name = "years"
employees_idx.stack()
freight_idx = freight_idees.groupby(["nuts1", "nuts2", "nuts3", "IDEES_category"]).sum(numeric_only=True)
# freight_idx

2008    2009    2010    2011  \
nuts1 nuts2 nuts3 IDEES_category                                                
AT1   AT11  AT111 Food, beverages and tobacco   186.0   231.0   143.0   230.0   
                  Machinery Equipment             2.0     4.0    19.0     8.0   
                  Mining and quarrying          456.0   572.0   245.0   604.0   
                  Other Industrial Sectors        5.0     9.0    46.0    19.0   
                  Textiles and leather           31.0    31.0    31.0    31.0   
...                                               ...     ...     ...     ...   
UKN   UKN0  UKN05 Mining and quarrying         7005.0  6448.0  4270.0  3711.0   
                  Other Industrial Sectors      101.0    96.0   106.0     8.0   
                  Textiles and leather          113.0    75.0    42.0     5.0   
                  Transport Equipment            49.0    36.0    43.0    49.0   
                  Wood and wood products        775.0  1021.0   856.0   308.0   

                                                 2012    2013    2014    2015  \
nuts1 nuts2 nuts3 IDEES_category                                                
AT1   AT11  AT111 Food, beverages and tobacco   249.0   310.0   203.0   367.0   
                  Machinery Equipment             5.0    12.0     2.0     9.0   
                  Mining and quarrying          639.0   448.0   334.0   516.0   
                  Other Industrial Sectors       16.0    40.0    68.0    57.0   
                  Textiles and leather           31.0    31.0    31.0    31.0   
...                                               ...     ...     ...     ...   
UKN   UKN0  UKN05 Mining and quarrying         3016.0  6083.0  4121.0  4783.0   
                  Other Industrial Sectors       47.0    27.0    62.0    24.0   
                  Textiles and leather           51.0    99.0    44.0    36.0   
                  Transport Equipment           141.0    36.0    41.0    79.0   
                  Wood and wood products        342.0   560.0   211.0   397.0   

                                                 2016    2017    2018    2019  
nuts1 nuts2 nuts3 IDEES_category                                               
AT1   AT11  AT111 Food, beverages and tobacco   372.0   303.0   403.0   261.0  
                  Machinery Equipment             4.0     2.0     1.0     1.0  
                  Mining and quarrying          622.0   540.0   658.0   493.0  
                  Other Industrial Sectors       40.0     3.0    16.0    10.0  
                  Textiles and leather           31.0    31.0    31.0    31.0  
...                                               ...     ...     ...     ...  
UKN   UKN0  UKN05 Mining and quarrying         4078.0  3120.0  3120.0  3120.0  
                  Other Industrial Sectors       26.0    62.0    62.0    62.0  
                  Textiles and leather           72.0    58.0    58.0    58.0  
                  Transport Equipment            75.0    95.0    95.0    95.0  
                  Wood and wood products        304.0   775.0   775.0   775.0  

[9131 rows x 12 columns]

In [276]:
# Get the freight intensity of each reagion at nuts 3 level
freight_intensity = freight_idx / freight_idx.groupby("nuts2").sum()
# employees_idx.mul(freight_intensity)
employees_idx

2008     2009     2010     2011  \
nuts1 nuts2 IDEES_category                                                      
AT1   AT11  Chemicals Industry               379.0    382.0    375.0    390.0   
            Food, beverages and tobacco     2871.0   2830.0   2700.0   2764.0   
            Machinery Equipment             4396.0   4390.0   4354.0   4514.0   
            Metals                             0.0      0.0      0.0      0.0   
            Non-metallic mineral products    867.0    776.0    759.0    707.0   
...                                            ...      ...      ...      ...   
UKN   UKN0  Other Industrial Sectors       71846.0  54143.0  54125.0  53572.0   
            Pulp, paper and printing        3825.0   2929.0   2596.0   3415.0   
            Textiles and leather            3021.0   2596.0   3841.0   2548.0   
            Transport Equipment            10363.0  10682.0   9471.0   9375.0   
            Wood and wood products          3939.0   2737.0   2393.0   2547.0   

                                              2012     2013     2014     2015  \
nuts1 nuts2 IDEES_category                                                      
AT1   AT11  Chemicals Industry               416.0    415.0    420.0    293.0   
            Food, beverages and tobacco     2960.0   2970.0   3069.0   3195.0   
            Machinery Equipment             4768.0   4648.0   4527.0   4408.0   
            Metals                             0.0      0.0      0.0      0.0   
            Non-metallic mineral products    730.0    800.0    857.0    860.0   
...                                            ...      ...      ...      ...   
UKN   UKN0  Other Industrial Sectors       50138.0  46266.0  46864.0  47711.0   
            Pulp, paper and printing        3442.0   3369.0   3266.0   3448.0   
            Textiles and leather            2608.0   2234.0   3241.0   2521.0   
            Transport Equipment             9980.0  10523.0  10458.0  11325.0   
            Wood and wood products          2329.0   2452.0   2250.0   2787.0   

                                              2016     2017     2018  
nuts1 nuts2 IDEES_category                                            
AT1   AT11  Chemicals Industry               353.0    417.0    452.0  
            Food, beverages and tobacco     3056.0   3259.0   3282.0  
            Machinery Equipment             4420.0   4479.0   4647.0  
            Metals                             0.0      0.0      0.0  
            Non-metallic mineral products    867.0    919.0    888.0  
...                                            ...      ...      ...  
UKN   UKN0  Other Industrial Sectors       49400.0  55857.0  55857.0  
            Pulp, paper and printing        4078.0   3554.0   3554.0  
            Textiles and leather            3274.0   3346.0   3346.0  
            Transport Equipment            11152.0  10494.0  10494.0  
            Wood and wood products          3093.0   3070.0   3070.0  

[3440 rows x 11 columns]

In [233]:
employees[(employees['nuts2'] == 'AT11') &  (employees['IDEES_category'] == "Food, beverages and tobacco")]

,NACE_code,indicator,nuts2,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,IDEES_category,nuts1
16014,C10,V16110,AT11,2540.0,2530.0,2329.0,2485.0,2395.0,2352.0,2388.0,2269.0,2235.0,2370.0,2437.0,"Food, beverages and tobacco",AT1
18635,C11,V16110,AT11,742.0,729.0,727.0,710.0,674.0,618.0,572.0,495.0,465.0,460.0,434.0,"Food, beverages and tobacco",AT1
21253,C12,V16110,AT11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"Food, beverages and tobacco",AT1


In [240]:
test = employees_idx.reset_index()
test[(test['nuts2'] == 'AT11') &  (test['IDEES_category'] == "Food, beverages and tobacco") & (test["years"] == 2018)]

,nuts1,nuts2,IDEES_category,years,0
3673,AT1,AT11,"Food, beverages and tobacco",2018,2437.0
7457,AT1,AT11,"Food, beverages and tobacco",2018,434.0
11142,AT1,AT11,"Food, beverages and tobacco",2018,0.0


In [124]:
proxy = pd.DataFrame(columns=["IDEES_category", "nuts2"] + years)

# employee-freight proxy
nuts2_intersect = set(freight_match.nuts2) & set(employees.nuts2)
freight_intersect = freight_match[freight_match.nuts2.isin(nuts2_intersect)]
# freight_intersect_nuts2_total = freight_intersect.groupby(["IDEES_category", "nuts2"]).sum(numeric_only=True)
# freight_nuts3_ratios = freight_intersect.set_index(["IDEES_code", "nuts2"]) / (freight_intersect_nuts2_total)
# freight_nuts3_ratios.reset_index()
freight_intersect

,NST_2007_code,units,nuts3,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,nuts2,nuts1
0,GT01,THS_T,AT111,129.0,156.0,180.0,99.0,98.0,212.0,81.0,128.0,53.0,450.0,297.0,79.0,AT11,AT1
1,GT01,THS_T,AT112,740.0,554.0,536.0,514.0,521.0,632.0,577.0,518.0,428.0,685.0,629.0,453.0,AT11,AT1
2,GT01,THS_T,AT113,161.0,255.0,143.0,184.0,171.0,98.0,166.0,266.0,171.0,185.0,184.0,76.0,AT11,AT1
3,GT01,THS_T,AT121,1114.0,1075.0,1187.0,935.0,716.0,757.0,753.0,692.0,747.0,721.0,735.0,741.0,AT12,AT1
4,GT01,THS_T,AT122,388.0,545.0,469.0,483.0,358.0,376.0,289.0,502.0,517.0,440.0,449.0,496.0,AT12,AT1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36211,TOTAL,THS_T,SK023,12681.0,19021.0,20196.0,15210.0,12644.0,11971.0,10701.0,11610.0,15332.0,19029.0,14391.0,17368.0,SK02,SK0
36212,TOTAL,THS_T,SK031,18265.0,18975.0,19056.0,16772.0,14913.0,13134.0,10563.0,14993.0,10325.0,13855.0,16287.0,26500.0,SK03,SK0
36213,TOTAL,THS_T,SK032,15362.0,11105.0,10910.0,11396.0,11525.0,12388.0,10288.0,8762.0,10833.0,10624.0,13258.0,14169.0,SK03,SK0
36214,TOTAL,THS_T,SK041,18403.0,14627.0,12422.0,10620.0,11119.0,13111.0,11596.0,10681.0,12343.0,13955.0,18482.0,21672.0,SK04,SK0


In [ ]:
# Combine freight and employee data
freight_employees = pd.concat([
    freight_eu['freight'].reset_index('nuts3'),
    industry_employees.reindex(freight_eu.droplevel('nuts3').index).to_frame('employees_nuts2')
], axis=1).set_index('nuts3', append=True)

freight_employees = freight_employees.reset_index()
employees = industry_employees.reset_index()

In [ ]:
freight_employees.sort_values(['subsector', 'year', 'nuts2', 'nuts3'])